In [18]:
# Install required packages for Bengali RAG system with OCR
# Run this cell first to install all necessary packages
# !pip install langchain-openai
# !pip install langchain-community
# !pip install fastembed
# !pip install chromadb
# !pip install sentence-transformers
# !pip install pymupdf  # For PDF to image conversion
# !pip install pytesseract  # OCR engine
# !pip install Pillow  # Image processing
# !pip install opencv-python  # Image preprocessing
# !pip install numpy

# Note: You also need to install Tesseract OCR separately:
# Download from: https://github.com/UB-Mannheim/tesseract/wiki
# Make sure to install Bengali language data (ben.traineddata)

In [19]:
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.schema import Document
import sys
import os
import re
import fitz  # PyMuPDF for PDF to image
import pytesseract
from PIL import Image
import cv2
import numpy as np
from typing import List, Optional, Tuple

In [ ]:
# Set up OpenAI API key
# You can get your API key from: https://platform.openai.com/api-keys
openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key

In [21]:
# Configure Tesseract OCR
# Make sure you have installed Tesseract and Bengali language data
# Download Tesseract: https://github.com/UB-Mannheim/tesseract/wiki
# Bengali language data should be in tessdata folder

# Set Tesseract path (adjust according to your installation)
# For Windows: 
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# For Linux/Mac, it's usually in PATH, so you might not need to set this
# pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Test if Bengali is available
try:
    available_langs = pytesseract.get_languages()
    if 'ben' in available_langs:
        print("✅ Bengali language support is available in Tesseract")
    else:
        print("❌ Bengali language support not found. Please install Bengali traineddata.")
        print("Download ben.traineddata from: https://github.com/tesseract-ocr/tessdata")
except Exception as e:
    print(f"⚠️ Tesseract configuration issue: {e}")
    print("Please make sure Tesseract is properly installed and configured.")

✅ Bengali language support is available in Tesseract


In [22]:
def preprocess_image_for_ocr(image_array: np.ndarray) -> np.ndarray:
    """
    Preprocess image for better OCR results
    """
    # Convert to grayscale if needed
    if len(image_array.shape) == 3:
        gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    else:
        gray = image_array
    
    # Increase contrast and brightness
    alpha = 1.2  # Contrast control
    beta = 10    # Brightness control
    adjusted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(adjusted, (1, 1), 0)
    
    # Apply threshold to get binary image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to clean up the image
    kernel = np.ones((1, 1), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    return cleaned

def pdf_page_to_image(pdf_path: str, page_num: int, dpi: int = 300) -> Image.Image:
    """
    Convert a specific PDF page to high-resolution image
    """
    doc = fitz.open(pdf_path)
    page = doc[page_num]
    
    # Create transformation matrix for high DPI
    mat = fitz.Matrix(dpi/72, dpi/72)
    
    # Render page to pixmap
    pix = page.get_pixmap(matrix=mat)
    
    # Convert to PIL Image
    img_data = pix.tobytes("ppm")
    image = Image.open(io.BytesIO(img_data))
    
    doc.close()
    return image

def extract_text_with_ocr(pdf_path: str, page_num: int, dpi: int = 300) -> str:
    """
    Extract text from PDF page using OCR
    """
    try:
        # Convert PDF page to image
        image = pdf_page_to_image(pdf_path, page_num, dpi)
        
        # Convert PIL image to numpy array
        img_array = np.array(image)
        
        # Preprocess image for better OCR
        processed_img = preprocess_image_for_ocr(img_array)
        
        # Convert back to PIL Image for pytesseract
        pil_image = Image.fromarray(processed_img)
        
        # OCR configuration for Bengali
        custom_config = r'--oem 3 --psm 6 -l ben+eng'  # Bengali + English
        
        # Extract text using OCR
        text = pytesseract.image_to_string(pil_image, config=custom_config)
        
        return text.strip()
        
    except Exception as e:
        print(f"Error extracting text from page {page_num}: {e}")
        return ""

def preprocess_bengali_text(text: str) -> str:
    """
    Preprocess Bengali text for better processing
    """
    if not text:
        return ""
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove common OCR artifacts
    text = re.sub(r'[^\u0980-\u09FF\u0020-\u007F\u2000-\u206F\u2E00-\u2E7F]', ' ', text)
    
    # Clean up punctuation spacing
    text = re.sub(r'\s+([।,;:!?])', r'\1', text)
    text = re.sub(r'([।,;:!?])\s*', r'\1 ', text)
    
    return text.strip()

In [23]:
import io  # For BytesIO

In [24]:
def ingest_bengali_documents_with_ocr(
    pdf_path: str = "Data/HSC26-Bangla1st-Paper.pdf",
    start_page: Optional[int] = None,
    end_page: Optional[int] = None,
    dpi: int = 300,
    embedding_model: str = "text-embedding-3-small"  # OpenAI embedding model
):
    """
    Ingest Bengali PDF documents using OCR and create vector store with OpenAI embeddings
    
    Args:
        pdf_path: Path to the PDF file
        start_page: Starting page number (0-indexed). If None, starts from beginning
        end_page: Ending page number (0-indexed, inclusive). If None, goes to end
        dpi: DPI for image conversion (higher = better quality but slower)
        embedding_model: OpenAI embedding model to use
                        Options: "text-embedding-3-small", "text-embedding-3-large", "text-embedding-ada-002"
    """
    
    # Open PDF to get page count
    doc = fitz.open(pdf_path)
    total_pages = len(doc)
    doc.close()
    
    # Set page range
    start_page = start_page if start_page is not None else 0
    end_page = end_page if end_page is not None else total_pages - 1
    
    # Validate page range
    start_page = max(0, min(start_page, total_pages - 1))
    end_page = max(start_page, min(end_page, total_pages - 1))
    
    print(f"Processing pages {start_page} to {end_page} (total: {end_page - start_page + 1} pages)")
    print(f"PDF has {total_pages} total pages")
    print(f"Using OpenAI embedding model: {embedding_model}")
    
    # Extract text from each page using OCR
    documents = []
    
    for page_num in range(start_page, end_page + 1):
        print(f"Processing page {page_num + 1}/{total_pages}...")
        
        # Extract text using OCR
        text = extract_text_with_ocr(pdf_path, page_num, dpi)
        
        if text.strip():  # Only add if text was extracted
            # Preprocess Bengali text
            cleaned_text = preprocess_bengali_text(text)
            
            if cleaned_text:  # Only add if cleaning didn't remove everything
                # Create document with metadata
                doc = Document(
                    page_content=cleaned_text,
                    metadata={
                        "source": pdf_path,
                        "page": page_num + 1,  # 1-indexed for user display
                        "extraction_method": "OCR",
                        "dpi": dpi,
                        "embedding_model": embedding_model
                    }
                )
                documents.append(doc)
                print(f"✅ Page {page_num + 1}: Extracted {len(cleaned_text)} characters")
            else:
                print(f"⚠️ Page {page_num + 1}: No valid text after cleaning")
        else:
            print(f"❌ Page {page_num + 1}: No text extracted")
    
    if not documents:
        print("❌ No text was extracted from any pages!")
        return None
    
    print(f"\n📄 Total documents created: {len(documents)}")
    
    # Split the documents with optimized settings for Bengali
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Slightly larger chunks for OpenAI embeddings
        chunk_overlap=200,  # More overlap for context preservation
        length_function=len,
        add_start_index=True,
        separators=[
            "\n\n",  # Paragraph breaks
            "\n",    # Line breaks
            "।",     # Bengali sentence end (dari)
            "?",     # Question mark
            "!",     # Exclamation
            ".",     # Period
            " ",     # Space
            "",      # Character level
        ],
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"📝 Split {len(documents)} documents into {len(chunks)} chunks.")
    
    # Use OpenAI embeddings
    print(f"🔄 Creating embeddings using {embedding_model}...")
    embedding = OpenAIEmbeddings(model=embedding_model)
    
    # Create vector store
    vector_store = Chroma.from_documents(
        documents=chunks, 
        embedding=embedding, 
        persist_directory="./bengali_chroma_db_openai"  # Different directory for OpenAI embeddings
    )
    
    print("✅ Bengali document ingestion with OCR and OpenAI embeddings completed!")
    return vector_store

In [25]:
# Run this once to create the vector store for Bengali documents using OCR and OpenAI embeddings
# You can specify page ranges to process only specific pages

# Example 1: Process all pages with default embedding model
# ingest_bengali_documents_with_ocr("Data/HSC26-Bangla1st-Paper.pdf")

# Example 2: Process pages 5 to 10 with specific embedding model
# ingest_bengali_documents_with_ocr(
#     "Data/HSC26-Bangla1st-Paper.pdf", 
#     start_page=4, 
#     end_page=9,
#     embedding_model="text-embedding-3-large"  # More powerful but more expensive
# )

# Example 3: Process from page 7 to 18 with high DPI and small embedding model
ingest_bengali_documents_with_ocr(
    pdf_path="Data/HSC26-Bangla1st-Paper.pdf",
    start_page=42,    # Start from page 7 (0-indexed)
    end_page=49,     # End at page 18 (0-indexed, inclusive)
    dpi=400,         # Higher DPI for better OCR quality
    embedding_model="text-embedding-3-small"  # Fast and cost-effective
)

Processing pages 42 to 48 (total: 7 pages)
PDF has 49 total pages
Using OpenAI embedding model: text-embedding-3-small
Processing page 43/49...
✅ Page 43: Extracted 1906 characters
Processing page 44/49...
✅ Page 43: Extracted 1906 characters
Processing page 44/49...
✅ Page 44: Extracted 1429 characters
Processing page 45/49...
✅ Page 44: Extracted 1429 characters
Processing page 45/49...
✅ Page 45: Extracted 2208 characters
Processing page 46/49...
✅ Page 45: Extracted 2208 characters
Processing page 46/49...
✅ Page 46: Extracted 2221 characters
Processing page 47/49...
✅ Page 46: Extracted 2221 characters
Processing page 47/49...
✅ Page 47: Extracted 2218 characters
Processing page 48/49...
✅ Page 47: Extracted 2218 characters
Processing page 48/49...
✅ Page 48: Extracted 1430 characters
Processing page 49/49...
✅ Page 48: Extracted 1430 characters
Processing page 49/49...
✅ Page 49: Extracted 1068 characters

📄 Total documents created: 7
📝 Split 7 documents into 24 chunks.
🔄 Creatin

In [26]:
# Utility function to preview OCR results before full processing
def preview_ocr_extraction(
    pdf_path: str = "Data/HSC26-Bangla1st-Paper.pdf", 
    page_num: int = 0, 
    dpi: int = 300
):
    """
    Preview OCR extraction results for a specific page
    """
    print(f"🔍 Previewing OCR extraction for page {page_num + 1}")
    print("=" * 60)
    
    # Extract text using OCR
    raw_text = extract_text_with_ocr(pdf_path, page_num, dpi)
    cleaned_text = preprocess_bengali_text(raw_text)
    
    print(f"📄 Raw text length: {len(raw_text)} characters")
    print(f"🧹 Cleaned text length: {len(cleaned_text)} characters")
    print("\n" + "=" * 60)
    print("📝 Raw OCR Output (first 500 chars):")
    print("-" * 40)
    print(raw_text[:500])
    print("\n" + "=" * 60)
    print("✨ Cleaned Text (first 500 chars):")
    print("-" * 40)
    print(cleaned_text[:500])
    
    return raw_text, cleaned_text

# Test OCR on first page
preview_ocr_extraction("Data/HSC26-Bangla1st-Paper.pdf", page_num=42, dpi=300)

🔍 Previewing OCR extraction for page 43
📄 Raw text length: 1914 characters
🧹 Cleaned text length: 1912 characters

📝 Raw OCR Output (first 500 chars):
----------------------------------------
1911
প্রশ্থ- ২: পড়াশুনা শেষ করে সবিতা এখন গ্রামের একটি সরকারি প্রাইমারি স্কুলে শিক্ষকতা করেন। বছর
কয়েক আগে শহরের এক ধনী ব্যবসায়ীর ছেলের সাথে তার বিবাহ স্থির হয়। পাত্রপক্ষ বিয়েতে মোটা অঙ্কের
যৌতুক দাবি করলে তার আত্মসম্মানে আঘাত লাগে। সবিতা নিজেই যৌতুককে প্রত্যাখ্যান করে বিয়ে না করার
সিদ্ধান্তে অটল থাকেন। পিতামাতা ও সহকর্মীদের অনেক অনুরোধ সত্ত্বেও তিনি তার চিন্তা-চেতনায় কোনো
পরিবর্তন আনেননি। তিনি ছাত্র-ছাত্রীদের প্রাণ। মায়ের মতো ভালোবাসা দিয়ে আগলে রাখেন সবাইকে। তিনি
বলেন, "দেশকে মাতৃজ্ঞান

✨ Cleaned Text (first 500 chars):
----------------------------------------
1911 প্রশ্থ- ২: পড়াশুনা শেষ করে সবিতা এখন গ্রামের একটি সরকারি প্রাইমারি স্কুলে শিক্ষকতা করেন  বছর কয়েক আগে শহরের এক ধনী ব্যবসায়ীর ছেলের সাথে তার বিবাহ স্থির হয়  পাত্রপক্ষ বিয়েতে মোটা অঙ্কের যৌতুক দাবি করলে তার আত্মসম্মানে আঘাত

('1911\nপ্রশ্থ- ২: পড়াশুনা শেষ করে সবিতা এখন গ্রামের একটি সরকারি প্রাইমারি স্কুলে শিক্ষকতা করেন। বছর\nকয়েক আগে শহরের এক ধনী ব্যবসায়ীর ছেলের সাথে তার বিবাহ স্থির হয়। পাত্রপক্ষ বিয়েতে মোটা অঙ্কের\nযৌতুক দাবি করলে তার আত্মসম্মানে আঘাত লাগে। সবিতা নিজেই যৌতুককে প্রত্যাখ্যান করে বিয়ে না করার\nসিদ্ধান্তে অটল থাকেন। পিতামাতা ও সহকর্মীদের অনেক অনুরোধ সত্ত্বেও তিনি তার চিন্তা-চেতনায় কোনো\nপরিবর্তন আনেননি। তিনি ছাত্র-ছাত্রীদের প্রাণ। মায়ের মতো ভালোবাসা দিয়ে আগলে রাখেন সবাইকে। তিনি\nবলেন, "দেশকে মাতৃজ্ঞানে সেবা করা, দেশকে ভালোবাসা প্রত্যেকের কর্তব্য।" পরহিতে জীবন উৎসর্গ করাই\nতার ধর্ম।\n\n[ঢাকা বোর্ড: ২০২২]\nক. অনুপমের বন্ধু হরিশ কোথায় কাজ করে?\nখ. "এইটে একবার পরখ করিয়া দেখো।”- ব্যাখ্যা কর।\nগ. "উদ্দীপকের \'সবিতা\' ও "অপরিচিতা\' গল্পের \'কল্যাণী\' উভয়েই যৌতুকের শিকার।"- মন্তব্যটি\nবিশ্লেষণ কর।\nঘ. "সবিতার দেশপ্রেম কল্যাণীর মাতৃআজ্ঞার সাথে একই সূত্রে গাথা ।"- উক্তিটির যথার্থতা বিচার কর।\nসমাধান:\nক. অনুপমের বন্ধু হরিশ কানপুরে কাজ করে।\nখ. শস্তুনাথ সেন আলোচ্য উক্তির মধ্য দিয়ে একজোড়া এ

In [ ]:
def create_bengali_rag_chain(embedding_model: str = "text-embedding-3-small"):
    """
    Create RAG chain optimized for Bengali language using OpenAI embeddings
    
    Args:
        embedding_model: OpenAI embedding model to use for retrieval
    """
    # Use OpenAI GPT model with Bengali-optimized settings
    model = ChatOpenAI(
        model="gpt-4",  # GPT-4 has better multilingual support
        temperature=0.2,  # Lower temperature for more consistent Bengali responses
        max_tokens=1500   # More tokens for Bengali responses
    )
    
    # Bengali-optimized prompt template
    prompt = PromptTemplate.from_template(
        """
        আপনি একজন সহায়ক বাংলা ভাষার সহায়ক। শুধুমাত্র নিম্নলিখিত প্রসঙ্গের উপর ভিত্তি করে প্রশ্নের উত্তর দিন।
        যদি আপনি উত্তর না জানেন, তাহলে বলুন "এই প্রশ্নের জন্য কোনো প্রসঙ্গ পাওয়া যায়নি: {input}"।
        
        You are a helpful Bengali language assistant. Answer the question based only on the following context.
        If you don't know the answer, reply "No context available for this question: {input}".
        
        প্রশ্ন / Question: {input}
        প্রসঙ্গ / Context: {context}
        
        উত্তর / Answer:
        """
    )

    print(prompt)
    
    # Load vector store with OpenAI embeddings
    print(f"🔄 Loading vector store with OpenAI embeddings ({embedding_model})...")
    embedding = OpenAIEmbeddings(model=embedding_model)
    vector_store = Chroma(
        persist_directory="./bengali_chroma_db_openai", 
        embedding_function=embedding
    )

    # Create retriever with optimized settings for Bengali
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 5,  # Retrieve more documents for better context
            "score_threshold": 0.1,  # Lower threshold for OpenAI embeddings (they're more precise)
        },
    )

    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)
    
    return chain

In [28]:
def ask_in_bengali(query: str, embedding_model: str = "text-embedding-3-small"):
    """
    Ask questions in Bengali or English and get responses using OpenAI embeddings
    
    Args:
        query: The question to ask
        embedding_model: OpenAI embedding model to use for retrieval
    """
    # Preprocess the query
    processed_query = preprocess_bengali_text(query)
    
    # Create the chain with specified embedding model
    chain = create_bengali_rag_chain(embedding_model=embedding_model)
    
    # Invoke chain
    result = chain.invoke({"input": processed_query})
    
    # Print results
    print("উত্তর / Answer:")
    print("-" * 50)
    print(result["answer"])
    print("\n" + "=" * 50)
    print("সূত্র / Sources:")
    print("-" * 50)
    
    for i, doc in enumerate(result["context"], 1):
        print(f"{i}. Source: {doc.metadata['source']}")
        if 'page' in doc.metadata:
            print(f"   Page: {doc.metadata['page']}")
        if 'embedding_model' in doc.metadata:
            print(f"   Embedding Model: {doc.metadata['embedding_model']}")
        print(f"   Content preview: {doc.page_content[:100]}...")
        print()
    
    return result

In [ ]:
# # Additional utility function to search similar documents using OpenAI embeddings
# def search_similar_bengali_content(query: str, k: int = 3, embedding_model: str = "text-embedding-3-small"):
#     """
#     Search for similar Bengali content without generating answers using OpenAI embeddings
    
#     Args:
#         query: Search query
#         k: Number of results to return
#         embedding_model: OpenAI embedding model to use
#     """
#     embedding = OpenAIEmbeddings(model=embedding_model)
#     vector_store = Chroma(
#         persist_directory="./bengali_chroma_db_openai", 
#         embedding_function=embedding
#     )
    
#     docs = vector_store.similarity_search(query, k=k)
    
#     print(f"অনুসন্ধানের ফলাফল / Search Results for: '{query}'")
#     print(f"Using OpenAI embedding model: {embedding_model}")
#     print("=" * 60)
    
#     for i, doc in enumerate(docs, 1):
#         print(f"\n{i}. Source: {doc.metadata['source']}")
#         if 'page' in doc.metadata:
#             print(f"   Page: {doc.metadata['page']}")
#         if 'embedding_model' in doc.metadata:
#             print(f"   Embedding Model: {doc.metadata['embedding_model']}")
#         print(f"   Content: {doc.page_content[:200]}...")
#         print("-" * 40)
    
#     return docs

In [ ]:
# Example: Search for similar content using OpenAI embeddings
# search_similar_bengali_content("আমি আশা", k=3, embedding_model="text-embedding-3-small")

অনুসন্ধানের ফলাফল / Search Results for: 'আমি আশা'
Using OpenAI embedding model: text-embedding-3-small

1. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 17
   Embedding Model: text-embedding-3-small
   Content: ? না, কোনো কালেই না  আমার মনে আছে, কেবল সেই এক রাত্রির অজানা কণ্ঠের মধুর সুরের আশা-জায়গা আছে  নিশ্চয়ই আছে  নইলে দীড়াব কোথায়  তাই বৎসরের পর বৎসর যায় আমি এইখানেই আছি  দেখা হয়, সেই কণ্ঠ শুনি, যখন স...
----------------------------------------

2. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 16
   Embedding Model: text-embedding-3-small
   Content: “না, আপনি যাইতে পারিবেন না, যেমন আছেন বসিয়া থাকুন ” করা, এ কথা মিথ্যা কথা ” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয়া দিল  করা, এ কথা মিথ্যা কথা ” বলিয়া নাম লেখা টিকিটটি খুলিয়া...
----------------------------------------

3. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 17
   Embedding Model: text-embedding-3-small
   Content: মামার নিষেধ অমান্য করিয়া, মাতৃ-আজ্ঞা ঠেলিয়া, তার পরে আমি কানপুরে আসিয়াছি  

[Document(metadata={'dpi': 400, 'embedding_model': 'text-embedding-3-small', 'extraction_method': 'OCR', 'page': 17, 'source': 'Data/HSC26-Bangla1st-Paper.pdf', 'start_index': 825}, page_content='? না, কোনো কালেই না  আমার মনে আছে, কেবল সেই এক রাত্রির অজানা কণ্ঠের মধুর সুরের আশা-জায়গা আছে  নিশ্চয়ই আছে  নইলে দীড়াব কোথায়  তাই বৎসরের পর বৎসর যায় আমি এইখানেই আছি  দেখা হয়, সেই কণ্ঠ শুনি, যখন সুবিধা পাই কিছু তার কাজ করিয়া দিই - আর মন বলে, এই তো জায়গা পাইয়াছি  ওগো অপরিচিতা, তোমার পরিচয়ের শেষ হইল না, শেষ হইবে না; কিন্তু ভাগ্য আমার ভালো, এই তো আমি জায়গা পাইয়াছি  17'),
 Document(metadata={'dpi': 400, 'embedding_model': 'text-embedding-3-small', 'extraction_method': 'OCR', 'page': 16, 'source': 'Data/HSC26-Bangla1st-Paper.pdf', 'start_index': 0}, page_content='“না, আপনি যাইতে পারিবেন না, যেমন আছেন বসিয়া থাকুন ” করা, এ কথা মিথ্যা কথা ” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয়া দিল  করা, এ কথা মিথ্যা কথা ” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয

In [37]:
# Ask question using OpenAI embeddings
ask_in_bengali("অনুপমের বন্ধু হরিশ কোথায় কাজ করে?", embedding_model="text-embedding-3-small")

🔄 Loading vector store with OpenAI embeddings (text-embedding-3-small)...
উত্তর / Answer:
--------------------------------------------------
"এই প্রশ্নের জন্য কোনো প্রসঙ্গ পাওয়া যায়নি: অনুপমের বন্ধু হরিশ কোথায় কাজ করে?"

সূত্র / Sources:
--------------------------------------------------
1. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 43
   Embedding Model: text-embedding-3-small
   Content preview: 19111] প্রশ্ন- ২: পড়াশুনা শেষ করে সবিতা এখন গ্রামের একটি সরকারি প্রাইমারি স্কুলে শিক্ষকতা করেন  বছর...

2. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 47
   Embedding Model: text-embedding-3-small
   Content preview: . উদ্দীপকের পরেশ 'অপরিচিতা' গল্পের অনুপম চরিত্রের বিপরীত  যথার্থ মূল্যবোধসম্পন্ন মানুষ কখনই অসংগতিকে...

3. Source: Data/HSC26-Bangla1st-Paper.pdf
   Page: 49
   Embedding Model: text-embedding-3-small
   Content preview: . 'সেই লগ্নে এসেছি পালিয়ে'- এ চরণের আলোকে উদ্দীপকের নায়কের মতো অনুপমের বিরহের জন্য নিজের অক্ষমতাই ...

4. Source: Data/HSC26-Bangla1st-Paper.pdf


{'input': 'অনুপমের বন্ধু হরিশ কোথায় কাজ করে?',
 'context': [Document(metadata={'dpi': 400, 'embedding_model': 'text-embedding-3-small', 'extraction_method': 'OCR', 'page': 43, 'source': 'Data/HSC26-Bangla1st-Paper.pdf', 'start_index': 0}, page_content='19111] প্রশ্ন- ২: পড়াশুনা শেষ করে সবিতা এখন গ্রামের একটি সরকারি প্রাইমারি স্কুলে শিক্ষকতা করেন  বছর কয়েক আগে শহরের এক ধনী ব্যবসায়ীর ছেলের সাথে তার বিবাহ স্থির হয়  পাত্রপক্ষ বিয়েতে মোটা অঙ্কের যৌতুক দাবি করলে তার আত্মসম্মানে আঘাত লাগে  সবিতা নিজেই যৌতুককে প্রত্যাখ্যান করে বিয়ে না করার সিদ্ধান্তে অটল থাকেন  পিতামাতা ও সহকর্মীদের অনেক অনুরোধ Wage তিনি তার চিন্তা-চেতনায় কোনো পরিবর্তন আনেননি  তিনি ছাত্র-ছাত্রীদের প্রাণ  মায়ের মতো ভালোবাসা দিয়ে আগলে রাখেন সবাইকে   তিনি বলেন, "দেশকে মাতৃজ্ঞানে সেবা করা, দেশকে ভালোবাসা প্রত্যেকের কর্তব্য " পরহিতে জীবন উৎসর্গ করাই তার ধর্ম  [ঢাকা বোর্ড: ২০২২] ক. অনুপমের বন্ধু হরিশ কোথায় কাজ করে'),
  Document(metadata={'dpi': 400, 'embedding_model': 'text-embedding-3-small', 'extraction_method': 'OCR', 

In [ ]:
# # 🔄 Compare different OpenAI embedding models
# def compare_openai_embedding_models(query: str):
#     """
#     Compare results from different OpenAI embedding models
#     """
#     models = [
#         "text-embedding-3-small",   # Fast and cost-effective
#         "text-embedding-3-large",   # More powerful but expensive
#         "text-embedding-ada-002"    # Previous generation, still good
#     ]
    
#     print(f"🔍 Comparing OpenAI embedding models for query: '{query}'")
#     print("=" * 80)
    
#     for model in models:
#         try:
#             print(f"\n📊 Results using {model}:")
#             print("-" * 50)
            
#             # Search for similar content
#             docs = search_similar_bengali_content(query, k=2, embedding_model=model)
            
#             if docs:
#                 print(f"✅ Found {len(docs)} results")
#             else:
#                 print("❌ No results found")
                
#         except Exception as e:
#             print(f"❌ Error with {model}: {e}")
        
#         print("\n" + "="*50)

# # Test comparison (uncomment to run)
# # compare_openai_embedding_models("কল্যাণী")